### <p style="text-align: right;"> &#9989; **Put your name here** </p>
#### <p style="text-align: right;"> &#9989; Put your group member names here</p>

# Day 17 In-Class Assignment: Machine Learning

<img src="http://blog.bidmotion.com/wp-content/uploads/sites/3/2016/06/supervised-workflow-machine-learning.png" width=700px>

1. **Scientific motivation** 
    - Face Recognition
2. **Modeling tools** 
    - Machine Learning (Support Vector Machines)
3. **Programming concepts** 
    - Discovering existing code
    - Some examples of reusable code
4. **Python Programming Concepts** 
    - Sci-kit learn
    
    

### Agenda for today's class

</p>

1. Review pre-class assignment
1. Problem Statement
1. Example using the digits dataset
1. Face Recognition

---
# 1. Review of Pre-Class assignment

We'll discussion any questions that came up as a class.

----
# 2. Problem Statement
What if we wanted to build a model to do face recognition? Ideally, the model would take an image as an input and return the name of the person represented in the image.

&#9989; **Do This:** As a group, discuss things you could do with a face recognition model and record the ideas that come up.

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and write down some example applications where face recognition may be useful.

#### Defining the features and building the model

If you review the image at the top of the notebook, you might notice that one of the first steps in machine learning is to go from "raw data" into a set of "features" and "labels", which we discussed a bit about when we worked with the perceptron model. Extracting features from our data can sometimes be one of the trickier parts of the process and also one of the most important ones. We have to think carefully about exactly what the "right" features are for training our machine learning algorithm and, when possible, it is advantageous to find ways to reduce the total number of features we are trying to model. Once we define our features, we can build our model.

&#9989; **Do This:** Now, also as a group, discuss and be prepared to share with the class how you would define a feature vector from an image and how you would build a face recognition model using machine learning.  Can you find any libraries that you think would be will suited for this? Which of these libraries might you actually use?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and put a basic outline of what your group discussed.

----
# 3. Example using the digits dataset

Let's start with downloading a dataset called "digits" which is included in the sklearn library. Pay special attention to the new library import that we're going to use! The `sklearn` module has *many* submodules.

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

import numpy as np
from sklearn.datasets import fetch_lfw_people, load_digits

sk_data = load_digits();

In [ ]:
#Cool slider to browse all of the images.
from ipywidgets import interact
def browse_images(images, labels, categories):
    n = len(images)
    def view_image(i):
        plt.imshow(images[i], cmap=plt.cm.gray_r, interpolation='nearest')
        plt.title('%s' % categories[labels[i]])
        plt.axis('off')
        plt.show()
    interact(view_image, i=(0,n-1))

In [ ]:
browse_images(sk_data.images, sk_data.target, sk_data.target_names)

Now let's inspect the digits arrays to find out what the shapes of the arrays (which can help for plotting the data with matplotlib). **Review the code below and make sure you know what it is doing.**

In [ ]:
feature_vectors = sk_data.data
class_labels = sk_data.target
categories = sk_data.target_names

n_samples, n_features = feature_vectors.shape
N, h, w = sk_data.images.shape
n_classes = len(categories)

**Question**: Write some code to print out the number of samples, number of features, number of classes, and the shape of the image dimensions:

In [ ]:
#Put your answer to the above question here


**Question**: As a group discuss the difference between the features, samples, and classes.  How do these relate to the shape of the image?  Write down a quick definition of each (the first one has been done for you):

 <font size=8 color="#009600">&#9998;</font> Do This - Erase this first line of this cell and finish describing variables. Use appropriate markdown to make your answer readable. (double-click on this text to edit this cell, and hit shift+enter to save the text)
 
 
1. **n_samples:** Total number of images in the digits dataset. 
2. **n_features:**
3. **n_classes:**
4. **N:**
5. **h:**
6. **w:**


#### Distribution of classes in our data

Let's have a look at the distribution of samples across the target classes:

In [ ]:
plt.figure(figsize=(14, 3))

y_unique = np.unique(class_labels)
counts = [(class_labels == i).sum() for i in y_unique]

plt.xticks(y_unique,  categories[y_unique])
locs, labels = plt.xticks()
plt.setp(labels, rotation=45, size=20)
_ = plt.bar(y_unique, counts)

**Question**: Does this seem like a good set of samples for training our machine learning algorithm? Why?

 <font size=8 color="#009600">&#9998;</font> Do This - Erase this first line of this cell and answer the above question. (double-click on this text to edit this cell, and hit shift+enter to save the text)

----
## Step A: Splitting the dataset for model into training and testing sets
Let's split the data in a training set and final testing set. We want to randomly select 75% of the data for training and 25% of the data for testing.  

&#9989; Do This - As a group, come up with a way to split the input data into separate training and testing sets.  Make sure you keep the feature vectors and classes together.  

**BIG HINT**: This is a very common step in machine learning, maybe there exists a function to do this for you? I bet you can find an example for this exact problem online.

In [ ]:
# Put your code here.  For consistency with the rest of the 
# code use the following variables.  

# NOTE This code is wrong.  Think about why you do not want to use all of 
# the feature_vectors for training and testing.

train_vectors = feature_vectors
test_vectors = feature_vectors
train_labels = class_labels
test_labels = class_labels

**Question**: Why do we need to separate our samples into a training and testing set. Why can't we just use all the data for both? Wouldn't that make it work better?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

----
## Step B: Train a SVM Classifier based on the training dataset.

The following code will train the Support Vector Machine (SVM).  The resulting model will be called ```clf``` for classifier.  Note that there is a lot is going on in these functions.  We will not get into all of the details, lets just accept the following values as reasonable starting points (actually, these values came from an example  found online):

In [ ]:
###############################################################################
# Train a SVM classification model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import time
start = time.time()

#make some temporary variables so you can change this easily
tmp_vectors = train_vectors
tmp_labels = train_labels

print("Fitting the classifier to the training set")
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(tmp_vectors, tmp_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

end = time.time()
print("Runtime",end - start)

&#9989; **DO THIS**: Explore the ```clf``` object. What functions does it have access to?  Can you figure out what function you may use to input a unknown feature vector and make a class prediction?

In [ ]:
## DO THIS, put your exploration code here.


----
## Step C. Show the results of the classification on the testing dataset.

In [ ]:
###############################################################################
# Quantitative evaluation of the model quality on the test set

#make some temporary variables so you can change this easily
predict_vectors = test_vectors
true_lables = test_labels

print("Predicting names on the test set")
pred_labels = clf.predict(predict_vectors)

print(classification_report(true_lables, pred_labels))
print(confusion_matrix(true_lables, pred_labels, labels=range(n_classes)))


In [ ]:
def plot_gallery(images, true_titles, pred_titles, h, w, n_row=5, n_col=5):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray_r)
        plt.title('Pred='+str(categories[pred_titles[i]]), size=9)
        plt.xlabel('Actual='+str(categories[true_titles[i]]), size=9)
        plt.xticks(())
        plt.yticks(())

plot_gallery(test_vectors, test_labels, pred_labels, h,w)

**Question:** How well is the classifier doing with the digits dataset? Comment on what information the classification report and confusion matrix provide you.

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

**Questions:** What if you created a new random training set from the images using the same fraction of images? What if you just used all of the data -- does it work better or worse? Why?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

# 4. Face Recognition

Now that we have completed the example for digits dataset. Lets do it again with some faces. Fortunately, scikit-learn comes with a face dataset in exactly the same format as the digits dataset.  This means we should just be able to swap out one with the other. Here is the code for importing the faces data.  This code ensures there are at least 50 faces per person and they are resized to 40%.  

```sk_data = fetch_lfw_people(min_faces_per_person=50, resize=0.4)```

&#9989; **DO THIS**:  Repeat the entire process using the face database imported with the command shown above. Answer the following questions.

In [ ]:
# copy the code above with the modified data set and paste it here

**Question:** How long did it take to train the face recognition classifier?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

**Question:** How well did the SVM algorithm work on the face recognition problem?  Can you think of real world applications where this level of face recognition may be acceptable?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

**Question:** Why is the face recognition not working as well?  

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

**Question:** Give some example science problems where this type of machine learning classification (SVM) may be used.

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

----
# Assignment Wrap-up

Fill out the following Google Form before submitting your assignment to D2L!

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://goo.gl/forms/noAwoJd9pEj7A1su1" 
	width="800px" 
	height="600px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

-----
### Congratulations, we're done!

Now, you just need to submit this assignment by uploading it to the course <a href="https://d2l.msu.edu/">Desire2Learn</a> web page for today's submission folder (Don't forget to add your names in the first cell).


&#169; Copyright 2018,  Michigan State University Board of Trustees